In [2]:
import numpy as np
import pandas as pd

In [2]:
url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vQjh5HzZ1N0SU7ME9ZQRzeVTaXaGsV97rU8R7eAcg53k27GTstJp9cRUOfr55go1GRRvTz1NwvyOnuh/pub?gid=1562145139&single=true&output=csv"
titanic_df = pd.read_csv(url)

In [3]:
titanic_df.head(1)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,NaN,S


### Q-1: Using groupby make groups using the "Pclass" column and find out the average age and total number of missing values in the "Age" column for every group.

In [7]:
obj = titanic_df.groupby("Pclass")
obj["Age"].mean()

Pclass
1    38.233441
2    29.877630
3    25.140620
Name: Age, dtype: float64

In [18]:
for i,group in obj:
    print(f"Missing values in group {i} is {group['Age'].isna().sum()}")

Missing values in group 1 is 30
Missing values in group 2 is 11
Missing values in group 3 is 136


### Q-2: Using groupby make groups using the "Pclass" column and fill every group's "Embarked" column's missing values with the mode value of that group. After that, print every group's "Embarked" column's value counts in ascending order.

In [29]:
for i,group in obj:
    group["Embarked"].fillna(group["Embarked"].mode()[0],inplace=True)
    print(group["Embarked"].value_counts().sort_values())

Q      2
C     85
S    129
Name: Embarked, dtype: int64
Q      3
C     17
S    164
Name: Embarked, dtype: int64
C     66
Q     72
S    353
Name: Embarked, dtype: int64


### Q-3: Make groups based on "Embarked" column. And for each of this embarked group, make another group based on "Pclass" and find out the average fare (round off up to 2 decimal places) for each "Pclass" for each group of "Embarked".

In [73]:
dict1 = {}
grp = titanic_df.groupby("Embarked")
for i in grp.groups.keys():
    Pclass_group = grp.get_group(i).groupby("Pclass")
    dict1[i] = {}
    for j in Pclass_group.groups.keys():
        dict1[i][j] = round(Pclass_group.get_group(j)["Fare"].mean(),2)
dict1        

{'C': {1: 104.72, 2: 25.36, 3: 11.21},
 'Q': {1: 90.0, 2: 12.35, 3: 11.18},
 'S': {1: 70.36, 2: 20.33, 3: 14.64}}

## Questions Based on Fifa Worldcup - 2022 Dataset:

#### You can read the dataset by using the below sample code

In [74]:
fifa_df = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vT3D_x_4DS6d51LKJ7ze1sxT5WpV5uiSVOFYHLwBiGru6vFyVv5h5-83AwFjxWYiWfCDjDAaarHAV-k/pub?gid=0&single=true&output=csv")

In [78]:
fifa_df.head(2)

,Sl. No,Match No.,Team,Against,Group,Goal,Possession (%),Inside Penalty Area,Outside Penalty Area,Assists,...,Fouls Against,Offsides,Passes,Passes Completed,Crosses,Crosses Completed,Corners,Free Kicks,Penalties Scored,Pts
0,1,1,Qatar,Ecuador,A,0,40,0,0,0,...,15,3,453,387,10,5,1,19,0,0
1,2,1,Ecuador,Qatar,A,2,46,2,0,1,...,15,4,484,419,26,10,3,17,1,3


#### Q-4: Perform groupby based on the "Team" column and then perform Z Normalization on top of the below columns of each group:
#### Passes
#### Passes Completed
#### Attempted Line Breaks
#### Completed Line Breaks
#### You have to make a python function named z_normalization which takes two arguments:

#### group: Every group that you have created
#### cols_to_perform: This parameter takes a list of columns on which you have to perform the Z-Normalization.
#### For this problem, you have to use th apply() method.

In [94]:
grp_obj = fifa_df.groupby("Team")
list_col = ["Passes","Passes Completed" , "Attempted Line Breaks","Completed Line Breaks"]

In [95]:
def z_normaliation(group,col_name):
    for i in col_name:
        avg = group[i].mean()
        std = group[i].std()
        z_norm = (group[i] - avg)/std
        group["Z-Norm"] = z_norm
    return group

In [101]:
from warnings import filterwarnings
filterwarnings("ignore")
df = grp_obj.apply(z_normaliation,list_col)
df

,Sl. No,Match No.,Team,Against,Group,Goal,Possession (%),Inside Penalty Area,Outside Penalty Area,Assists,...,Offsides,Passes,Passes Completed,Crosses,Crosses Completed,Corners,Free Kicks,Penalties Scored,Pts,Z-Norm
0,1,1,Qatar,Ecuador,A,0,40,0,0,0,...,3,453,387,10,5,1,19,0,0,-1.153113
1,2,1,Ecuador,Qatar,A,2,46,2,0,1,...,4,484,419,26,10,3,17,1,3,0.817624
2,3,2,England,Iran,B,6,69,6,0,6,...,2,810,733,29,9,8,16,0,3,-1.334904
3,4,2,Iran,England,B,2,20,2,0,1,...,2,232,156,11,3,0,10,1,0,-1.147648
4,5,3,Senegal,Netherlands,A,0,39,0,0,0,...,2,391,326,22,8,6,14,0,0,-0.355371
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,124,62,Morocco,Fram,F,0,55,0,0,0,...,3,583,518,22,1,3,15,0,0,0.899236
124,125,63,Croatia,Morocco,F,2,45,2,0,2,...,2,491,430,21,3,6,13,0,3,-0.464907
125,126,63,Morocco,Croatia,F,1,45,1,0,0,...,2,494,428,20,5,3,15,0,0,1.079083
126,127,64,Argentina,France,C,3,46,3,0,1,...,4,648,544,20,4,6,22,5,3,1.808140


In [106]:
grp_obj = fifa_df.groupby("Team")
grp_obj.agg({
    "Passes" : [min,max],
    "Yellow Cards": [min,max,"mean"],
    "Attempted Line Breaks" : ["min","max","std"],
    "Possession (%)" : "mean"
})

Passes       Yellow Cards               Attempted Line Breaks  \
                  min   max          min max      mean                   min   
Team                                                                           
Argentina         408   862            0   8  2.285714                   141   
Australia         286   466            0   3  1.750000                   133   
Belgium           512   685            1   3  1.666667                   167   
Brazil            548   695            0   3  1.200000                   164   
Cameroon          295   500            1   5  2.666667                   144   
Canada            448   536            2   4  2.666667                   102   
Costa Rica        231   454            1   3  2.000000                    86   
Croatia           461   724            0   2  1.142857                    97   
Denmark           537   650            1   2  1.666667                   173   
Ecuador           429   484            0   2  1.000000                   143   
England           508   810            0   1  0.200000                   133   
France            358   734            0   3  1.142857                   126   
Germany           352   827            0   3  1.000000                   143   
Ghana             319   460            2   4  2.666667                   122   
Iran              232   463            2   3  2.333333                    56   
Japan             225   591            0   3  1.500000                   104   
Korea Republic    359   548            1   2  1.500000                   139   
Mexico            356   500            1   4  2.333333                   127   
Morocco           229   583            0   3  1.285714                   128   
Netherlands       403   786            0   4  1.800000                   168   
Poland            303   466            1   3  1.750000                    79   
Portugal          461   678            0   3  1.200000                   165   
Qatar             424   457            0   4  2.333333                   159   
Saudi Arabia      267   560            2   6  4.666667                   147   
Senegal           277   511            1   3  1.750000                   143   
Serbia            403   513            2   7  4.000000                   140   
Spain             647  1070            0   1  0.500000                   186   
Switzerland       403   512            1   4  2.250000                   157   
Tunisia           328   501            1   3  1.666667                   153   
United States     427   572            0   4  1.250000                   143   
Uruguay           390   528            1   3  2.000000                   146   
Wales             325   508            1   2  1.666667                   104   

                               Possession (%)  
                max        std           mean  
Team                                           
Argentina       249  38.774685      49.285714  
Australia       171  17.682383      31.250000  
Belgium         195  14.422205      49.000000  
Brazil          193  10.691118      50.400000  
Cameroon        182  19.857828      38.333333  
Canada          176  37.753587      44.333333  
Costa Rica      154  38.974351      27.333333  
Croatia         259  54.499891      47.428571  
Denmark         241  34.000000      51.333333  
Ecuador         177  17.009801      42.666667  
England         208  29.676590      57.200000  
France          179  21.407609      44.857143  
Germany         256  56.624494      53.000000  
Ghana           188  33.020196      37.000000  
Iran            195  77.526340      31.666667  
Japan           253  64.048810      30.250000  
Korea Republic  201  27.072434      42.250000  
Mexico          198  40.706265      46.666667  
Morocco         157  10.641339      34.285714  
Netherlands     198  12.759310      44.000000  
Poland          156  36.326070      32.000000  
Portugal        197  13.049904      53.800000  
Qatar         

# Questions on IPL wala dataset

In [4]:
ipl = pd.read_csv("D:\Jupyter\ipl_deliveries.csv")
ipl.head(2)

,ID,Team,innings,overs,ballnumber,batter,bowler,non-striker,extra_type,batsman_run,extras_run,total_run,non_boundary,isWicketDelivery,player_out,kind,fielders_involved,BattingTeam,BowlingTeam
0,1312200,Rajasthan RoyalsGujarat Titans,1,0,1,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals,Gujarat Titans
1,1312200,Rajasthan RoyalsGujarat Titans,1,0,2,YBK Jaiswal,Mohammed Shami,JC Buttler,legbyes,0,1,1,0,0,NaN,NaN,NaN,Rajasthan Royals,Gujarat Titans


### Q-5: Find batsman in below category-
#### Highest score while chasing
#### Best Strike rate while chasing and have faced 100+ balls
#### Chasing mean team batting in second inning

In [126]:
#Highest score while chasing
chasing_df = ipl[ipl["innings"] == 2]
chasing_df.groupby(["ID","batter"])["batsman_run"].sum().sort_values(ascending=False).head().reset_index()

,ID,batter,batsman_run
0,501206,PC Valthaty,120
1,501243,V Sehwag,119
2,1254061,SV Samson,119
3,1136620,SR Watson,117
4,336018,ST Jayasuriya,114


In [148]:
#Best Strike rate while chasing and have faced 100+ balls
temp_df = chasing_df[chasing_df["extra_type"].isna()]
ob = temp_df.groupby("batter")
work_df = ob.agg({
    "ballnumber" : "count",
    "batsman_run" : "sum"
})
temp1 = work_df[work_df["ballnumber"] > 100]
temp1["Strike Rate"] = (temp1["batsman_run"] / temp1["ballnumber"])*100
temp1.reset_index().sort_values("Strike Rate",ascending=False).head(5)

,batter,ballnumber,batsman_run,Strike Rate
112,PJ Cummins,109,220,201.834862
146,SP Narine,340,597,175.588235
82,LS Livingstone,104,182,175.000000
5,AD Russell,557,974,174.865350
165,V Sehwag,845,1420,168.047337


### Q-6 Most Successful bowler against any batsman. Find that pair of bowler and batsman.
### Most Successful in terms of dissmissal. A bowler who have dissmissed any batsman most no of times. If any two pairs have same no of dissmisal, consider runs conceded by bowler to that batsman. Those who have concede lesser runs is more successful.

In [27]:
ipl.groupby(["bowler","batter"],as_index=False)[["isWicketDelivery","batsman_run"]].sum().sort_values(["isWicketDelivery","batsman_run"],ascending =[False,True]).head(1)

,bowler,batter,isWicketDelivery,batsman_run
20710,SP Narine,RG Sharma,8,137


### Q-7: Most successful batting pair in IPL. Batting pair who have scored most runs playing together.

In [173]:
ipl[["batter","non-striker"]]

,batter,non-striker
0,YBK Jaiswal,JC Buttler
1,YBK Jaiswal,JC Buttler
2,JC Buttler,YBK Jaiswal
3,YBK Jaiswal,JC Buttler
4,YBK Jaiswal,JC Buttler
...,...,...
225949,P Kumar,SB Joshi
225950,SB Joshi,P Kumar
225951,P Kumar,SB Joshi
225952,SB Joshi,P Kumar


In [195]:
def fun(x):
    return "-".join(list(np.sort(x.values)))
   
ipl["batter-pair"] = ipl[["batter","non-striker"]].apply(fun,axis=1)    
ipl.head(2)

,ID,Team,innings,overs,ballnumber,batter,bowler,non-striker,extra_type,batsman_run,extras_run,total_run,non_boundary,isWicketDelivery,player_out,kind,fielders_involved,BattingTeam,BowlingTeam,batter-pair
0,1312200,Rajasthan RoyalsGujarat Titans,1,0,1,YBK Jaiswal,Mohammed Shami,JC Buttler,NaN,0,0,0,0,0,NaN,NaN,NaN,Rajasthan Royals,Gujarat Titans,JC Buttler-YBK Jaiswal
1,1312200,Rajasthan RoyalsGujarat Titans,1,0,2,YBK Jaiswal,Mohammed Shami,JC Buttler,legbyes,0,1,1,0,0,NaN,NaN,NaN,Rajasthan Royals,Gujarat Titans,JC Buttler-YBK Jaiswal


In [199]:
ipl.groupby("batter-pair")["total_run"].sum().sort_values(ascending=False).head(5)

batter-pair
AB de Villiers-V Kohli    3134
CH Gayle-V Kohli          2802
DA Warner-S Dhawan        2357
G Gambhir-RV Uthappa      1906
KL Rahul-MA Agarwal       1731
Name: total_run, dtype: int64

### Q-8: Make a dataframe for all batting pairs played together.

In [ ]:
Batsman1 Batsman2 Runs Avg StrikeRate

#### Just to ease this question you can count wide-balls for strike rate.

In [229]:
df = ipl.groupby("batter-pair").agg({
    "total_run" : "sum",
    "ballnumber": "count",
    "isWicketDelivery" :"sum"
}).reset_index()
df["Batsman1"] = df["batter-pair"].apply(lambda x :x.split("-")[0])
df["Batsman2"] = df["batter-pair"].apply(lambda x :x.split("-")[1])
df["Strike-rate"] = (df["total_run"]/df["ballnumber"])*100
df["Avg"] = df["total_run"] / df["isWicketDelivery"]
df.sort_values("total_run",ascending=False,inplace=True)

In [230]:
df.rename( columns= {
    "total_run" : "Runs",
    "ballnumber" : "balls-played"
}
,inplace=True)

In [231]:
df[["Batsman1","Batsman2","Runs","Avg","Strike-rate"]]

,Batsman1,Batsman2,Runs,Avg,Strike-rate
302,AB de Villiers,V Kohli,3134,44.140845,152.209811
1251,CH Gayle,V Kohli,2802,52.867925,142.017233
1508,DA Warner,S Dhawan,2357,48.102041,136.637681
1954,G Gambhir,RV Uthappa,1906,39.708333,133.754386
2803,KL Rahul,MA Agarwal,1731,52.454545,142.939719
...,...,...,...,...,...
350,AD Mascarenhas,Gurkeerat Singh,0,0.000000,0.000000
3552,NLTC Perera,RV Gomez,0,0.000000,0.000000
2270,Iqbal Abdulla,TS Mills,0,0.000000,0.000000
3556,NM Coulter,Nile,0,0.000000,0.000000
